In [81]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [82]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

In [83]:
!pip install -qq Cython contextlib2 pillow lxml matplotlib pycocotools

In [84]:
from __future__ import division, print_function, absolute_import

import pandas as pd
import numpy as np
import csv

import re
import os
import io
import glob
import shutil
import urllib.request
import tarfile
import xml.etree.ElementTree as ET

import tensorflow.compat.v1 as tf
import cv2 

from PIL import Image
from collections import namedtuple, OrderedDict

from google.colab import files

In [85]:
print(tf.__version__)

2.3.0


In [86]:
%cd /gdrive/MyDrive/object_detection/data/

/gdrive/MyDrive/object_detection/data


In [6]:
ls

sample_data/


In [7]:
for label_path in ['train_labels', 'test_labels']:
  print(label_path)

train_labels
test_labels


In [87]:

#adjusted from: https://github.com/datitran/raccoon_dataset

#converts the annotations/labels into one csv file for each training and testing labels
#creats label_map.pbtxt file

%cd /gdrive/MyDrive/object_detection/data/


# images extension
images_extension = 'jpg'

# takes the path of a directory that contains xml files and converts
#  them to one csv file.

# returns a csv file that contains: image name, width, height, class, xmin, ymin, xmax, ymax.
# note: if the xml file contains more than one box/label, it will create more than one row for the same image. each row contains the info for an individual box. 
def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      classes_names.append(member[0].text)
      value = (root.find('filename').text ,
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               int(member[4][0].text),
               int(member[4][1].text),
               int(member[4][2].text),
               int(member[4][3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name) 
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

# for both the train_labels and test_labels csv files, it runs the xml_to_csv() above.
for label_path in ['train_labels', 'test_labels']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

# Creating the `label_map.pbtxt` file
label_map_path = os.path.join("label_map.pbtxt")

pbtxt_content = ""

#creats a pbtxt file the has the class names.
for i, class_name in enumerate(classes):
    # display_name is optional.
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n    display_name: 'plants'\n }}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)

/gdrive/MyDrive/object_detection/data
Successfully converted train_labels xml to csv.
Successfully converted test_labels xml to csv.


In [88]:
!cat label_map.pbtxt

item {
    id: 1
    name: 'plants'
    display_name: 'plants'
 }

In [89]:
!ls -l

total 8039
drwx------ 2 root root    4096 Nov 19 05:00 'Annotated plant images'
-rw------- 1 root root    8188 Nov 20 06:24 'Copy of train_labels.csv'
-rw------- 1 root root      65 Nov 20 06:44  label_map.pbtxt
-rw------- 1 root root       0 Nov 19 08:38  label_map_sample.pbtxt
drwx------ 3 root root    4096 Nov 20 04:16  models
drwx------ 2 root root    4096 Nov 20 00:28  plant_images
drwx------ 2 root root    4096 Nov 19 05:18  test_labels
-rw------- 1 root root    2568 Nov 20 06:44  test_labels.csv
-rw------- 1 root root       0 Nov 19 23:55  test_labels.record
drwx------ 2 root root    4096 Nov 19 05:18  train_labels
-rw------- 1 root root    8188 Nov 20 06:44  train_labels.csv
-rw------- 1 root root 8182751 Nov 20 06:42  train_labels.record
-rw------- 1 root root    8172 Nov 20 05:27  train_labelsTest.csv


In [91]:
#checks if the images box position is placed within the image.

#note: while this doesn't checks if the boxes/annotatoins are correctly
# placed around the object, Tensorflow will through an error if this occured.
%cd /gdrive/MyDrive/object_detection/data/
# path to images
images_path = 'plant_images'

#loops over both train_labels and test_labels csv files to do the check
# returns the image name where an error is found 
# return the incorrect attributes; xmin, ymin, xmax, ymax.
for CSV_FILE in ['train_labels.csv', 'test_labels.csv']:
  with open(CSV_FILE, 'r') as fid:  
      print('[*] Checking file:', CSV_FILE) 
      file = csv.reader(fid, delimiter=',')
      first = True 
      cnt = 0
      error_cnt = 0
      error = False
      for row in file:
          if error == True:
              error_cnt += 1
              error = False         
          if first == True:
              first = False
              continue     
          cnt += 1      
          name, width, height, xmin, ymin, xmax, ymax = row[0], int(row[1]), int(row[2]), int(row[4]), int(row[5]), int(row[6]), int(row[7])     
          path = os.path.join(images_path, name)
          img = cv2.imread(path)         
          if type(img) == type(None):
              error = True
              print('Could not read image', name, img)
              continue     
          org_height, org_width = img.shape[:2]     
          if org_width != width:
              error = True
              print('Width mismatch for image: ', name, width, '!=', org_width)     
          if org_height != height:
              error = True
              print('Height mismatch for image: ', name, height, '!=', org_height) 
          if xmin > org_width:
              error = True
              print('XMIN > org_width for file', name)  
          if xmax > org_width:
              error = True
              print('XMAX > org_width for file', name)
          if ymin > org_height:
              error = True
              print('YMIN > org_height for file', name)
          if ymax > org_height:
              error = True
              print('YMAX > org_height for file', name)
          if error == True:
              print('Error for file: %s' % name)
              print()
      print()
      print('Checked %d files and realized %d errors' % (cnt, error_cnt))
      print("-----")

/gdrive/MyDrive/object_detection/data
[*] Checking file: train_labels.csv
Could not read image 47.jpg None
Could not read image 47.jpg None
Could not read image 47.jpg None
Could not read image 47.jpg None
Could not read image 47.jpg None
Could not read image 47.jpg None

Checked 208 files and realized 6 errors
-----
[*] Checking file: test_labels.csv

Checked 64 files and realized 0 errors
-----


In [92]:
import csv
print(csv.__file__)

/usr/lib/python3.6/csv.py


In [59]:
# rm plant_images/'47.jpg'

rm: cannot remove 'plant_images/47.jpg': No such file or directory


In [93]:
#removing the entry for it in the csv for that image as well

#because we did a random split for the data, we dont know if it ended up being in training or testing
# we will remove the image from both.

#training
#reading the training csv
df = pd.read_csv('/gdrive/MyDrive/object_detection/data/train_labels.csv')
# removing 47.jpg
df = df[df['filename'] != '47.jpg']
#reseting the index
df.reset_index(drop=True, inplace=True)
#saving the df
df.to_csv('/gdrive/MyDrive/object_detection/data/train_labels.csv')


#testing
#reading the testing csv
df = pd.read_csv('/gdrive/MyDrive/object_detection/data/test_labels.csv')
# removing 47.jpg
df = df[df['filename'] != '47.jpg']
#reseting the index
df.reset_index(drop=True, inplace=True)
#saving the df
df.to_csv('/gdrive/MyDrive/object_detection/data/test_labels.csv')

# Just for the memory
df = None


## Don't change the bottom part

In [35]:
# #adjusted from: https://github.com/datitran/raccoon_dataset
# def xml_to_csv(path):
#   classes_names = []
#   xml_list = []

#   for xml_file in glob.glob(path + '/*.xml'):
#     tree = ET.parse(xml_file)
#     root = tree.getroot()
#     for member in root.findall('object'):
#       classes_names.append(member[0].text)
#       value = (root.find('filename').text + '.jpg',
#                int(root.find('size')[0].text),
#                int(root.find('size')[1].text),
#                member[0].text,
#                int(member[4][0].text),
#                int(member[4][1].text),
#                int(member[4][2].text),
#                int(member[4][3].text))
#       xml_list.append(value)
#   column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
#   xml_df = pd.DataFrame(xml_list, columns=column_name) 
#   classes_names = list(set(classes_names))
#   classes_names.sort()
#   return xml_df, classes_names

# for label_path in ['train_labels', 'test_labels']:
#   image_path = os.path.join(os.getcwd(), label_path)
#   xml_df, classes = xml_to_csv(label_path)
#   xml_df.to_csv(f'{label_path}.csv', index=None)
#   print(f'Successfully converted {label_path} xml to csv.')

# label_map_path = os.path.join("label_map.pbtxt")
# pbtxt_content = ""

# for i, class_name in enumerate(classes):
#     pbtxt_content = (
#         pbtxt_content
#         + "item {{\n    id: {0}\n    name: '{1}'\n}}\n\n".format(i + 1, class_name)
#     )
# pbtxt_content = pbtxt_content.strip()
# with open(label_map_path, "w") as f:
#     f.write(pbtxt_content)

Successfully converted train_labels xml to csv.
Successfully converted test_labels xml to csv.


In [ ]:
# xml_to_csv('/gdrive/My Drive/object_detection/data/test_labels')

(      filename  width  height   class  xmin  ymin  xmax  ymax
 0    1.jpg.jpg    595     890  plants    10    36   595   571
 1    5.jpg.jpg   2048    1365  plants  1404   615  2048  1351
 2    5.jpg.jpg   2048    1365  plants   119   457   393  1365
 3    5.jpg.jpg   2048    1365  plants   778   230  1393  1351
 4    5.jpg.jpg   2048    1365  plants  1683    54  1821   621
 ..         ...    ...     ...     ...   ...   ...   ...   ...
 59  11.jpg.jpg   1600    1200  plants     4   314   447   480
 60  11.jpg.jpg   1600    1200  plants   883   565   988   642
 61  11.jpg.jpg   1600    1200  plants  1039   531  1191   667
 62  11.jpg.jpg   1600    1200  plants  1509   565  1598   729
 63  11.jpg.jpg   1600    1200  plants  1452   306  1600   431
 
 [64 rows x 8 columns], ['plants'])

In [ ]:
# xml_to_csv('/gdrive/My Drive/object_detection/data/train_labels')

(       filename  width  height   class  xmin  ymin  xmax  ymax
 0    14.jpg.jpg    898    1600  plants    20   322   523   798
 1    14.jpg.jpg    898    1600  plants   530   512   898   984
 2    13.jpg.jpg   1200    1600  plants     5   632    88   846
 3    13.jpg.jpg   1200    1600  plants   942   592  1089   881
 4    13.jpg.jpg   1200    1600  plants   470   557   701   971
 ..          ...    ...     ...     ...   ...   ...   ...   ...
 203  49.jpg.jpg    826     683  plants   272   128   351   269
 204  49.jpg.jpg    826     683  plants     1     2   214   461
 205  49.jpg.jpg    826     683  plants   634   230   826   302
 206  55.jpg.jpg    275     183  plants    12    40   150   125
 207  55.jpg.jpg    275     183  plants   152     1   275   150
 
 [208 rows x 8 columns], ['plants'])

In [ ]:
import tensorflow as tf
print("GPU Available: ", tf.test.is_gpu_available())

print("Tensorflow Version: ", tf.__version__)
print("Keras Version: ", tf.keras.__version__)

#check GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU Available:  True
Tensorflow Version:  2.3.0
Keras Version:  2.4.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
!nvidia-smi

Thu Nov 19 23:54:26 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W /  70W |    227MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [23]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [24]:
!protoc --version

libprotoc 3.0.0


In [25]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /gdrive/My Drive/object_detection/data/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1599128 sha256=cc8eab5cf0979c3534992b29c02f628dda08f4bca28466570faa488d001d5e66
  Stored in directory: /tmp/pip-ephem-wheel-cache-4t_c0dfd/wheels/21/08/f5/4d77e68b51c0bb44926b8748766b236ededc4b2f9631c82aa2
  Created wheel for avro-python3: filename=avro_python3-1.10.0-cp36-none-any.whl size=43735 sha256=44af59f46ac0d74833bdffb80e0d31964a223f3f83261614d786267b4d97065c
  Stored in directory: /root/.cache/pip/wheels/3f/15/cd/fe4ec8b88c130393464703ee8111e2cddebdc40e1b59ea85e9
  Created wheel for dill: filename=dill-0.3.1.1-cp36-none-any.whl size=78532 sha256=38a54e738a7886163a58f92d99cb7f769384c402cee468146481ea4037a23c33
  Stored in directory: /root/.cache/pip/wheels/59/b1/91/f02e76c732915c4015ab4010f3015469866c1eb9b14058d8e7
  Created wheel for hdfs: filename=hdfs-2.5.8-cp36-none-any.whl size=33213 sha256=f873935ed2d392b3e03d7f45bb87

ERROR: multiprocess 0.70.10 has requirement dill>=0.3.2, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.25.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1; python_version >= "3.0", but you'll have avro-python3 1.10.0 which is incompatible.


In [26]:
!python ./models/research/object_detection/builders/model_builder_tf2_test.py

2020-11-20 06:10:37.757881: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model
2020-11-20 06:10:40.416190: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-20 06:10:40.469734: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-20 06:10:40.470357: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-11-20 06:10:40.470428: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully o

In [29]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder


%matplotlib inline

In [94]:

#adjusted from: https://github.com/datitran/raccoon_dataset
from object_detection.utils import dataset_util


#change this to the base directory where your data/ is 
data_base_url = '/gdrive/MyDrive/object_detection/data/'

#location of images
image_dir = data_base_url +'plant_images/'

def class_text_to_int(row_label):
  if row_label == 'plants':
    return 1
  else:
    None

def split(df, group):
  data = namedtuple('data', ['filename', 'object'])
  gb = df.groupby(group)
  return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
	with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
	  encoded_jpg = fid.read()
	encoded_jpg_io = io.BytesIO(encoded_jpg)
	image = Image.open(encoded_jpg_io)
	width, height = image.size
	filename = group.filename.encode('utf8')
	image_format = b'jpg'
	xmins = []
	xmaxs = []
	ymins = []
	ymaxs = []
	classes_text = []
	classes = []

	for index, row in group.object.iterrows():
		xmins.append(row['xmin'] / width)
		xmaxs.append(row['xmax'] / width)
		ymins.append(row['ymin'] / height)
		ymaxs.append(row['ymax'] / height)
		classes_text.append(row['class'].encode('utf8'))
		classes.append(class_text_to_int(row['class']))

	tf_example = tf.train.Example(features=tf.train.Features(feature={
		'image/height': dataset_util.int64_feature(height),
		'image/width': dataset_util.int64_feature(width),
		'image/filename': dataset_util.bytes_feature(filename),
		'image/source_id': dataset_util.bytes_feature(filename),
		'image/encoded': dataset_util.bytes_feature(encoded_jpg),
		'image/format': dataset_util.bytes_feature(image_format),
		'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
		'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
		'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
		'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
		'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
		'image/object/class/label': dataset_util.int64_list_feature(classes),
	}))
	return tf_example
#creates tfrecord for both csv's
# for csv in ['train_labels', 'test_labels']:
#     writer = tf.io.TFRecordWriter(data_base_url + csv + '.record')
#     path = os.path.join(image_dir)
#     examples = pd.read_csv(data_base_url + csv + '.csv')
#     grouped = split(examples, 'filename')
#     for group in grouped:
#       tf_example = create_tf_example(group, path)
#       writer.write(tf_example.SerializeToString())
    
# writer.close()
# output_path = os.path.join(os.getcwd(), data_base_url + csv + '.record')
# print('Successfully created the TFRecords: {}'.format(data_base_url +csv + '.record'))

for csv in ['train_labels', 'test_labels']:
  writer = tf.io.TFRecordWriter(data_base_url + csv + '.record')
  path = os.path.join(image_dir)
  examples = pd.read_csv(data_base_url + csv + '.csv')
  grouped = split(examples, 'filename')
  for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())
  writer.close()
  output_path = os.path.join(os.getcwd(), data_base_url + csv + '.record')
  print('Successfully created the TFRecords: {}'.format(data_base_url +csv + '.record'))



Successfully created the TFRecords: /gdrive/MyDrive/object_detection/data/train_labels.record
Successfully created the TFRecords: /gdrive/MyDrive/object_detection/data/test_labels.record
